In [1]:
import os
import sys
# Change directory to two levels up
os.chdir("../..")
print(f"Current working directory: {os.getcwd()}")

Current working directory: /Users/lucapagano/Developer/embedNbreakfast


---

In [2]:
DATA_DIR='/Users/lucapagano/Developer/RecSys/ACM 2025/new_ubc_data/' # Product Properties
DATA_INPUT_DIR='/Users/lucapagano/Developer/RecSys/ACM 2025/new_ubc_data/input/' # Relevant Client
RESULT_DIR='gru_preprocessing/results'

In [4]:
!ls

README.md         gru_preprocessing synerise
assets            models


In [ ]:
import polars as pl
import numpy as np
from datetime import timedelta
from gru_preprocessing.util_functions import collapse, get_entropy_from_counts, add_nearby_skus, parse_embedding

data_dir = DATA_DIR
data_input_dir = DATA_INPUT_DIR
relevant_clients=np.load(data_input_dir+"relevant_clients.npy")

search_query=pl.read_parquet(data_input_dir+"search_query.parquet")

page_visit=pl.read_parquet(data_input_dir+"page_visit.parquet")

url_counts=(page_visit.group_by("url").agg(pl.len().alias("score")))
page_visit = page_visit.join(url_counts, on="url")


product_events=pl.read_parquet(f"{RESULT_DIR}/product_events.parquet")
top_skus = np.load(f"{RESULT_DIR}/top_skus.npy")
top_cat = np.load(f"{RESULT_DIR}/top_cats.npy")


## Page visits

In [ ]:
#applied only for final submission, to have all relevant clietns embeddings
filtered_visits=page_visit.filter(pl.col("client_id").is_in(relevant_clients))
exclueded_clients=relevant_clients[~np.isin(relevant_clients,np.concatenate([np.array(filtered_visits["client_id"].unique()),np.array(search_query["client_id"].unique()),np.array(product_events["client_id"].unique())]))]
filtered_visits=pl.concat([filtered_visits,page_visit.filter(pl.col("client_id").is_in(exclueded_clients) )])
filtered_visits

In [ ]:
#adding counts of nearby sku events
page_sessions=collapse(filtered_visits,"url")
page_sessions=add_nearby_skus(page_sessions,"url",product_events[["sku","client_id","timestamp","category"]],top_skus, top_cats)

c:\Users\vgius\Desktop\recsys2025pulito\recsys2025-main\ernesto\jijioriginal\util_functions.py:109: UserWarning: Sortedness of columns cannot be checked when 'by' groups provided
  joined = product_events_filtered.join_asof(


### Url features

In [39]:
url_counts = (
    page_visit
    .group_by("url")
    .agg(pl.len().alias("count"))
)
client_visit_total = (
    page_visit
    .group_by("client_id")
    .agg(pl.len().alias("client_total_visits"))
)
client_url_visits = (
    page_visit
    .group_by(["client_id","url"])
    .agg(pl.len().alias("client_url_visits"))
)
page_sessions=(page_sessions.join(url_counts,on="url",how="left")
            .join(client_visit_total,on="client_id",how="left")
 .join(client_url_visits,on=["url","client_id"],how="left")

)
page_sessions=(page_sessions
    .join(get_entropy_from_counts(page_sessions,"url","url_entropy"),on="url",how="left")
    .join(get_entropy_from_counts(page_sessions,"url","client_entropy",user_entropy=True),on="client_id",how="left"))

In [ ]:
page_sessions=page_sessions.rename({"session_start":"timestamp"}).drop(["session_id","session_end","session_end_ext"])
page_sessions.write_parquet("splitted/onlyrel/visit_events_sub.parquet")
filtered_visits=filtered_visits.filter(pl.col("client_id").is_in(relevant_clients))
page_sessions.write_parquet("splitted/visit_events_sub.parquet")

### Scaling

In [ ]:
import math

url_entropies_col=[ 'url_entropy',
 'client_entropy']
url_counts=[ 'count',
 'client_total_visits',
 'client_url_visits','num_events_nearby']

max_urls=math.log2(page_sessions["url"].n_unique())
max_clients=math.log2(page_sessions["client_id"].n_unique())
for c in url_counts:
    max_val=page_sessions[c].max()
    page_sessions=page_sessions.with_columns([
    (pl.col(c).fill_null(0.0) / (max_val + 1e-9)).alias(c)
    ])

for c in url_entropies_col:
    if( c=='url_entropy'):
        k=max_clients
    else:
        k=max_urls
    page_sessions=page_sessions.with_columns([
    (pl.col(c).fill_null(0.0) / (k + 1e-9)).alias(c)
    ])

max_time_diff=page_sessions["time_diff"].max()
page_sessions=page_sessions.with_columns([
    (pl.col('time_diff')/max_time_diff).alias('time_diff')
    ])

page_sessions

client_id,session_id,url,session_start,session_end,time_diff,session_end_ext,num_events_nearby,has_top_sku,has_top_category,count,client_total_visits,client_url_visits,url_entropy,client_entropy
i64,i64,i64,datetime[ns],datetime[ns],f64,datetime[μs],f64,i8,i8,f64,f64,f64,f64,f64
1032454,156693,1793995,2022-07-13 12:40:05,2022-07-13 12:40:45,0.016667,2022-07-13 12:45:45,0.0,0,0,0.000026,0.006177,0.000456,0.08185,0.260062
11554285,1724827,4467320,2022-10-09 12:53:40,2022-10-09 12:53:40,0.0,2022-10-09 12:58:40,0.0,0,0,0.000552,0.000239,0.000456,0.279254,0.100912
6545516,975087,13766510,2022-07-20 17:16:00,2022-07-20 17:20:10,0.104167,2022-07-20 17:25:10,0.00232,0,0,0.000014,0.000797,0.001367,0.08185,7.2720e-8
173208,26022,4998343,2022-08-03 10:34:40,2022-08-03 10:34:40,0.0,2022-08-03 10:39:40,0.0,0,0,0.000036,0.145379,0.073382,7.4428e-8,0.264072
13227305,1970088,6112348,2022-09-15 08:36:20,2022-09-15 09:06:55,0.764583,2022-09-15 09:11:55,0.0,0,0,0.006301,0.007054,0.004558,0.430145,0.216295
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
22823738,3403248,15081668,2022-06-25 14:53:05,2022-06-25 14:53:30,0.010417,2022-06-25 14:58:30,0.0,0,0,0.000028,0.003905,0.001367,0.154924,0.196396
4293987,639318,3869076,2022-08-21 17:41:00,2022-08-21 18:20:55,0.997917,2022-08-21 18:25:55,0.0,0,0,0.000212,0.0053,0.001823,0.246112,0.217138
13503565,2012760,5021179,2022-07-29 08:43:30,2022-07-29 08:56:20,0.320833,2022-07-29 09:01:20,0.006961,0,1,0.000342,0.001036,0.003646,0.24907,7.2720e-8


In [ ]:
page_sessions=page_sessions.rename({"session_start":"timestamp"}).drop(["session_id","session_end","session_end_ext"])
page_sessions.write_parquet("splitted/onlyrel/visit_events_sub.parquet")
filtered_visits=filtered_visits.filter(pl.col("client_id").is_in(relevant_clients))
page_sessions.write_parquet("splitted/visit_events_sub.parquet")

# Search queries

In [ ]:
from sklearn.cluster import KMeans
import numpy as np
import polars as pl

unique_queries = search_query.select("query").unique().with_row_index(name="query_id")
search_query = search_query.join(unique_queries, on="query", how="left").drop("query")

In [11]:
queries = pl.read_parquet("all/queries.parquet")
queries

query_id,query,embedding,cluster_id,centroid
u32,str,list[i32],i32,list[f64]
0,"""[115 166 229 224 115 106 26 1…","[115, 166, … 238]",83,"[110.286982, 182.569549, … 160.773893]"
1,"""[185 59 86 185 185 216 55 …","[185, 59, … 201]",99,"[162.847807, 81.593043, … 178.892783]"
2,"""[ 92 92 92 148 148 123 92 …","[92, 92, … 92]",3,"[123.793256, 123.326593, … 123.424568]"
3,"""[232 23 232 86 171 226 230 2…","[232, 23, … 8]",44,"[187.683225, 76.370597, … 88.268226]"
4,"""[ 85 250 185 184 252 230 60 …","[85, 250, … 101]",71,"[187.101214, 172.257938, … 82.10064]"
…,…,…,…,…
2472868,"""[181 7 155 87 60 94 203 …","[181, 7, … 107]",65,"[174.163558, 61.376225, … 187.377659]"
2472869,"""[163 68 58 154 163 174 212 2…","[163, 68, … 20]",8,"[83.300798, 92.530963, … 71.967465]"
2472870,"""[109 77 31 109 241 197 241 1…","[109, 77, … 98]",25,"[80.713287, 152.52866, … 151.285629]"


### Clusterring

In [ ]:
queries = unique_queries.with_columns(
    pl.col("query").map_elements(parse_embedding, return_dtype=pl.List(pl.Int32)).alias("embedding")
)

query_embeddings = np.array(queries["embedding"].to_list())

k = 100
kmeans = KMeans(n_clusters=k, random_state=42)
cluster_ids = kmeans.fit_predict(query_embeddings)
centroids = kmeans.cluster_centers_

centroids_queries=centroids
queries = queries.with_columns([
    pl.Series("cluster_id", cluster_ids)
])

In [ ]:
#scores
queries_global_scores=(
    search_query
    .group_by("query_id")
    .agg(pl.count().alias("query_score"))
)
queries_user_scores=(
    search_query
    .group_by(["client_id","query_id"])
    .agg(pl.count().alias("query_user_score"))
)

In [ ]:
queries = pl.read_parquet("all/queries.parquet")

In [19]:
search_query=search_query.join(queries[["cluster_id","query_id"]],on="query_id",how="left")

In [14]:
#cluster scores
queries_made_by_user=(search_query.group_by("client_id").agg(pl.count().alias("queries_made_by_user"))
             )
cluster_made_by_user=(search_query.group_by(["cluster_id","client_id"]).agg(pl.count().alias("cluster_made_by_user"))
             )
cluster_score=(search_query.group_by("cluster_id").agg(pl.count().alias("cluster_score"))
             )

C:\Users\vgius\AppData\Local\Temp\ipykernel_32528\1567863214.py:2: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  queries_made_by_user=(search_query.group_by("client_id").agg(pl.count().alias("queries_made_by_user"))
C:\Users\vgius\AppData\Local\Temp\ipykernel_32528\1567863214.py:4: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  cluster_made_by_user=(search_query.group_by(["cluster_id","client_id"]).agg(pl.count().alias("cluster_made_by_user"))
C:\Users\vgius\AppData\Local\Temp\ipykernel_32528\1567863214.py:6: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  cluster_score=(search_query.group_by("cluster_id").agg(pl.count().alias("cluster_score"))


### Colapsing consequent queries (40 min max)

In [16]:
search_query=collapse(search_query,"query_id")
search_query=add_nearby_skus(search_query,"query_id", product_events,top_skus, top_cats)

c:\Users\vgius\Desktop\recsys2025pulito\recsys2025-main\ernesto\jijioriginal\util_functions.py:112: UserWarning: Sortedness of columns cannot be checked when 'by' groups provided
  joined = product_events_filtered.join_asof(


In [ ]:
search_query=search_query.join(queries[["cluster_id","query_id"]],on="query_id",how="left")

client_id,session_id,query_id,session_start,session_end,time_diff,session_end_ext,num_events_nearby,has_top_sku,has_top_category
i64,i64,u32,datetime[ns],datetime[ns],i64,datetime[μs],i32,i8,i8
15295015,1408305,1534133,2022-09-16 10:54:40,2022-09-16 10:56:30,110,2022-09-16 11:01:30,0,0,0
17489012,1611483,1534133,2022-08-30 15:31:50,2022-08-30 15:31:50,0,2022-08-30 15:36:50,0,0,0
18469966,1702776,158474,2022-07-15 07:47:50,2022-07-15 07:47:50,0,2022-07-15 07:52:50,0,0,0
1688714,156012,426368,2022-08-29 15:49:05,2022-08-29 15:49:05,0,2022-08-29 15:54:05,0,0,0
6732381,620916,787982,2022-10-12 10:16:25,2022-10-12 10:16:25,0,2022-10-12 10:21:25,1,0,1
…,…,…,…,…,…,…,…,…,…
7650815,706961,391897,2022-07-17 19:25:15,2022-07-17 19:25:59,44,2022-07-17 19:30:59,0,0,0
18203450,1677856,1534133,2022-09-28 07:43:35,2022-09-28 07:44:40,65,2022-09-28 07:49:40,0,0,0
7087064,653948,341517,2022-08-26 01:17:40,2022-08-26 01:36:00,1100,2022-08-26 01:41:00,4,0,1


In [20]:
#adding scores and entropies
search_query=(search_query
              .join(queries_global_scores,on="query_id",how="left")
              .join(queries_user_scores,on=["client_id","query_id"],how="left")
                .join(queries_made_by_user,on="client_id",how="left")
                .join(cluster_made_by_user,on=["client_id","cluster_id"],how="left")
                .join(cluster_score,on="cluster_id",how="left")
             )
search_query=(search_query
    .join(get_entropy_from_counts(search_query,"query_id","query_entropy"),on="query_id",how="left")
    .join(get_entropy_from_counts(search_query,"cluster_id","cluster_entropy"),on="cluster_id",how="left")
    .join(get_entropy_from_counts(search_query,"query_id","user_query_entropy",user_entropy=True),on="client_id",how="left")
    .join(get_entropy_from_counts(search_query,"cluster_id","user_cluster_entropy",user_entropy=True),on="client_id",how="left")
    )
search_query

client_id,session_id,query_id,session_start,session_end,time_diff,session_end_ext,num_events_nearby,has_top_sku,has_top_category,cluster_id,query_score,query_user_score,queries_made_by_user,cluster_made_by_user,cluster_score,query_entropy,cluster_entropy,user_query_entropy,user_cluster_entropy
i64,i64,u32,datetime[ns],datetime[ns],i64,datetime[μs],i32,i8,i8,i32,u32,u32,u32,u32,u32,f64,f64,f64,f64
15295015,1408305,1534133,2022-09-16 10:54:40,2022-09-16 10:56:30,110,2022-09-16 11:01:30,0,0,0,94,1479596,40,45,40,1552386,16.602693,16.705589,0.000001,0.000001
17489012,1611483,1534133,2022-08-30 15:31:50,2022-08-30 15:31:50,0,2022-08-30 15:36:50,0,0,0,94,1479596,26,26,26,1552386,16.602693,16.705589,0.000001,0.000001
18469966,1702776,158474,2022-07-15 07:47:50,2022-07-15 07:47:50,0,2022-07-15 07:52:50,0,0,0,54,4,1,1,1,86377,1.0,14.114207,0.000001,0.000001
1688714,156012,426368,2022-08-29 15:49:05,2022-08-29 15:49:05,0,2022-08-29 15:54:05,0,0,0,8,28,1,1,1,70668,1.584962,13.706026,0.000001,0.000001
6732381,620916,787982,2022-10-12 10:16:25,2022-10-12 10:16:25,0,2022-10-12 10:21:25,1,0,1,19,3065,1,1,1,75073,10.017457,13.868551,0.000001,0.000001
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
7650815,706961,391897,2022-07-17 19:25:15,2022-07-17 19:25:59,44,2022-07-17 19:30:59,0,0,0,49,14,2,5,2,74540,2.947703,13.708411,1.0,1.0
18203450,1677856,1534133,2022-09-28 07:43:35,2022-09-28 07:44:40,65,2022-09-28 07:49:40,0,0,0,94,1479596,16,16,16,1552386,16.602693,16.705589,0.000001,0.000001
7087064,653948,341517,2022-08-26 01:17:40,2022-08-26 01:36:00,1100,2022-08-26 01:41:00,4,0,1,84,20619,355,607,360,94129,9.518071,13.817571,3.085885,3.085885


In [21]:
search_query=search_query.rename({"session_start":"timestamp"})

In [ ]:
queries = queries.with_columns(
    pl.col("cluster_id")
      .map_elements(lambda cid: centroids[int(cid)].tolist())   # prende il centroide giusto
      .alias("centroid")                                        # nuova colonna
)

In [ ]:
search_query.write_parquet("noscale/split14/search_events_sub.parquet")
search_query = search_query.filter(pl.col("client_id").is_in(relevant_clients))
search_query.write_parquet("noscale/split14/onlyrel/search_events_sub.parquet")

### Scaling

In [53]:
import math
queries_entropies=[ 'query_entropy',
 'cluster_entropy',
 'user_query_entropy',
 'user_cluster_entropy']
queries_counts=[
 'num_events_nearby',
 'query_score',
 'query_user_score',
 'queries_made_by_user',
 'cluster_made_by_user',
 'cluster_score']
max_queries=math.log2(search_query["query_id"].n_unique())
max_clients=math.log2(search_query["client_id"].n_unique())
max_clusters=math.log2(search_query["cluster_id"].n_unique())

for c in queries_counts:
    max_val=search_query[c].max()
    search_query=search_query.with_columns([
    (pl.col(c).fill_null(0.0) / (max_val + 1e-9)).alias(c)
    ])

for c in queries_entropies:
    if( c in [ 'query_entropy','cluster_entropy']):
        k=max_clients
    else:
        if(c=='user_query_entropy'):
            k=max_queries
        else:
            k=max_clusters
    search_query=search_query.with_columns([
    (pl.col(c).fill_null(0.0) / (k + 1e-9)).alias(c)
    ])
max_time_diff=search_query["time_diff"].max()
search_query=search_query.with_columns([
    (pl.col('time_diff')/max_time_diff).alias('time_diff')
    ])

search_query

client_id,session_id,query_id,timestamp,session_end,time_diff,session_end_ext,num_events_nearby,has_top_sku,has_top_category,cluster_id,query_score,query_user_score,queries_made_by_user,cluster_made_by_user,cluster_score,query_entropy,cluster_entropy,user_query_entropy,user_cluster_entropy
i64,i64,u32,datetime[ns],datetime[ns],f64,datetime[μs],f64,i8,i8,i32,f64,f64,f64,f64,f64,f64,f64,f64,f64
21805907,1721855,651936,2022-08-13 13:13:10,2022-08-13 13:13:35,0.010417,2022-08-13 13:18:35,0.0,0,0,27,8.9290e-7,0.000632,0.026432,0.000629,0.05421,7.2260e-8,0.69326,0.205732,0.584628
8834292,698900,1660052,2022-06-27 13:46:00,2022-06-27 13:46:00,0.0,2022-06-27 13:51:00,0.0,0,0,11,0.00028,0.000632,0.000367,0.000629,0.047408,0.334976,0.685156,7.4422e-8,2.1693e-7
11055200,873783,754415,2022-08-05 19:42:20,2022-08-05 19:47:10,0.120833,2022-08-05 19:52:10,0.0,0,0,73,0.000826,0.001263,0.022026,0.001259,0.046406,0.404687,0.682162,0.109888,0.320309
18354589,1450601,1646003,2022-09-01 09:34:35,2022-09-01 09:34:55,0.008333,2022-09-01 09:39:55,0.0,0,0,58,0.000179,0.005685,0.052496,0.005664,0.133284,0.11289,0.732996,0.237906,0.644909
21054430,1662663,1045097,2022-08-03 08:26:50,2022-08-03 09:06:40,0.995833,2022-08-03 09:11:40,0.014388,0,0,15,0.000088,0.008212,0.017988,0.00944,0.041602,0.187238,0.671118,0.103274,0.30103
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
7256457,574517,1529500,2022-07-08 12:58:00,2022-07-08 13:03:00,0.125,2022-07-08 13:08:00,0.002398,0,0,32,0.000021,0.001895,0.004405,0.001888,0.037982,0.173446,0.657097,0.051637,0.150515
20265272,1601089,793270,2022-07-28 16:13:00,2022-07-28 16:50:15,0.93125,2022-07-28 16:55:15,0.004796,0,1,25,0.000006,0.000632,0.003304,0.003776,0.050373,0.050137,0.699337,7.4422e-8,2.1693e-7
9240386,730705,15109,2022-07-12 07:07:40,2022-07-12 07:07:40,0.0,2022-07-12 07:12:40,0.002398,0,1,94,0.000004,0.000632,0.000367,0.000629,0.05131,0.050137,0.68562,7.4422e-8,2.1693e-7


In [ ]:
#adding query embeddings
max_val_queries=(
    queries.explode("embedding")
      .select(pl.col("embedding").max())
)
queries=queries.with_columns([(pl.col("embedding")/max_val_queries).alias("norm")])

In [ ]:

# search_query = search_query.filter(pl.col("client_id").is_in(relevant_clients))
search_query.write_parquet("noscale/split14/search_events_sub.parquet")
search_query = search_query.filter(pl.col("client_id").is_in(relevant_clients))
search_query.write_parquet("noscale/split14/onlyrel/search_events_sub.parquet")